In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROOQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)

result=llm.invoke("Howdy")
result

AIMessage(content='Howdy back atcha. What can I help you with today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 37, 'total_tokens': 52, 'completion_time': 0.017045067, 'prompt_time': 0.002034271, 'queue_time': 0.242717812, 'total_time': 0.019079338}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f64c0c8-a9e1-466f-b40e-1556cfd9f869-0', usage_metadata={'input_tokens': 37, 'output_tokens': 15, 'total_tokens': 52})

In [2]:
def add(a: float, b: float):
    """Adds two numbers."""
    return a + b
def subtract(a: float, b: float):
    """Subtracts two numbers."""
    return a - b
def multiply(a: float, b: float):
    """Multiplies two numbers."""
    return a * b
def divide(a: float, b: float):
    """Divides two numbers."""
    return a / b

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode,tools_condition
from IPython.display import Image, display

#Bind tools
tools=[add,subtract,multiply,divide]
llm_tool=llm.bind_tools(tools)

# System message prompt
system_message = SystemMessage(content="You are a helpful assistant. You help do a simple arithmetic.")

# ---- Node Function ----
def perform_task(state: MessagesState):
    return {"messages":[llm_tool.invoke([system_message] + state["messages"])]}



# ---- Build the Graph ----
builder = StateGraph(MessagesState)

# Nodes 
builder.add_node("perform_task", perform_task)
builder.add_node("tools", ToolNode(tools))

# Edges
builder.add_edge(START, "perform_task")
builder.add_conditional_edges(
    "perform_task", 
    tools_condition,
    )
builder.add_edge("tools", "perform_task")


# Memory saver
memory = MemorySaver()

# Compile with interrupt_before if needed
graph = builder.compile(interrupt_before=["perform_task"], checkpointer=memory)

# Show
# display(Image(graph.get_graph().draw_mermaid_png()))


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [16]:
import uuid
# ---- thread config ----
config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())  # generates a new, unique thread ID
    }
}


# Initial task
task = {"messages": HumanMessage(content="Add 5 and 6.")}

# Streaming response
for event in graph.stream(task, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Add 5 and 6.


In [17]:
state=graph.get_state(config)
state

StateSnapshot(values={'messages': [HumanMessage(content='Add 5 and 6.', additional_kwargs={}, response_metadata={}, id='679e3e71-04a5-4f48-b53e-68e796315199')]}, next=('perform_task',), config={'configurable': {'thread_id': '7586901b-e761-4fc0-9a67-451a8008b6b6', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1286-6010-6528-8000-9c2e192bf12e'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-10-04T13:45:33.059195+00:00', parent_config={'configurable': {'thread_id': '7586901b-e761-4fc0-9a67-451a8008b6b6', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1286-600e-63c2-bfff-4d532adec402'}}, tasks=(PregelTask(id='f4067a4d-6bcb-5bc2-ea87-8d1c002b025f', name='perform_task', path=('__pregel_pull', 'perform_task'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [18]:
state.next

('perform_task',)

In [19]:
for event in graph.stream(None, config,stream_mode="values"):
    event["messages"][-1].pretty_print()
    

================================ Human Message =================================

Add 5 and 6.
================================== Ai Message ==================================
Tool Calls:
  add (bbceev1ms)
 Call ID: bbceev1ms
  Args:
    a: 5
    b: 6
================================= Tool Message =================================
Name: add

11.0


In [20]:
# Continue to end 
for event in graph.stream(None, config,stream_mode="values"):
    event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: add

11.0
================================== Ai Message ==================================

The result of adding 5 and 6 is 11.


In [21]:
#update/edit message
task = {"messages": HumanMessage(content="Add 5 and 7.")}
for event in graph.stream(task, config,stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Add 5 and 7.


In [22]:
graph.update_state(config,{"messages": [HumanMessage(content="No, please Add 3 and 5.")]})

{'configurable': {'thread_id': '7586901b-e761-4fc0-9a67-451a8008b6b6',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0a128c-4906-6548-8006-986cc08ef62c'}}

In [23]:
for event in graph.stream(None, config,stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

No, please Add 3 and 5.
================================== Ai Message ==================================
Tool Calls:
  add (mn4da6b4y)
 Call ID: mn4da6b4y
  Args:
    a: 3
    b: 5
================================= Tool Message =================================
Name: add

8.0


In [24]:
new_state=graph.get_state(config).values
for message in new_state['messages']:
    message.pretty_print()

================================ Human Message =================================

Add 5 and 6.
================================== Ai Message ==================================
Tool Calls:
  add (bbceev1ms)
 Call ID: bbceev1ms
  Args:
    a: 5
    b: 6
================================= Tool Message =================================
Name: add

11.0
================================== Ai Message ==================================

The result of adding 5 and 6 is 11.
================================ Human Message =================================

Add 5 and 7.
================================ Human Message =================================

No, please Add 3 and 5.
================================== Ai Message ==================================
Tool Calls:
  add (mn4da6b4y)
 Call ID: mn4da6b4y
  Args:
    a: 3
    b: 5
================================= Tool Message =================================
Name: add

8.0
